In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
train_df = pd.read_csv("nlp-getting-started/train.csv")
test_df = pd.read_csv("nlp-getting-started/test.csv")

#### Data quick look

In [3]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [4]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

## Building Vectors
我們將在這個筆記本中構建的模型背後的理論非常簡單：
每條推文中包含的單詞是它們是否與真正的災難有關的良好指標（這並不完全正確，但這是一個很好的起點）。

我們將使用scikit-learn的CountVectorizer來計算每條推文中的單詞，並將其轉化為我們的機器學習模型可以處理的資料。

In [5]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [6]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


The above tells us that:<br>

1. There are 54 unique words (or "tokens") in the first five tweets.<br>
2. The first tweet contains only some of those unique tokens - all of the non-zero counts above are the tokens that DO exist in the first tweet.

Now let's create vectors for all of our tweets.

In [7]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

test_vectors = count_vectorizer.transform(test_df["text"])

請注意，我們這裡沒有使用 .fit_transform( )。 僅使用 .transform( ) 可以確保訓練向量中的token是唯一對應到測試向量的 token，即訓練和測試向量使用相同的 token set。

## Model
正如我們上面提到的，我們認為每條推文中包含的單詞是衡量它們是否與真正的災難有關的良好指標。 
推文中特定單詞（或一組單詞）的存在可能會直接關聯到該推文是否真實。

我們在這裡假設的是線性的。 所以讓我們建立一個線性模型！

Our vectors are really big, so we want to push our model's weights toward 0 without completely discounting different words - ridge regression is a good way to do this.

In [8]:
clf = linear_model.RidgeClassifier()

讓我們測試一下我們的模型，看看它在訓練資料上做得如何。 
為此，我們將使用交叉驗證——我們對已知資料的一部分進行訓練，然後與其他資料一起驗證。 如果我們這樣做幾次（使用不同的部分），我們可以很好地瞭解特定模型或方法的執行情況。
這場比賽的指標是F1。

In [9]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59453669, 0.56455572, 0.64113893])

上述分數並不糟糕！ 看起來我們的假設在排行榜上得分約為0.65。 有很多方法可以對此進行潛在改進（TFIDF、LSA、LSTM/RNN，清單很長！）-給他們中的任何一個機會！

與此同時，讓我們對我們的訓練集進行預測，併為比賽建立一個提交材料。